In [1]:
!pip install --quiet git+https://github.com/boudinfl/pke.git@dc4d5f21e0ffe64c4df93c46146d29d1c522476b
!pip install --quiet flashtext==2.7

     |████████████████████████████████| 245kB 10.1MB/s 


In [2]:
import textwrap
text = """There is a lot of volcanic activity at divergent plate boundaries in the oceans. For example, many undersea volcanoes are found along the Mid-Atlantic Ridge. This is a divergent plate boundary that runs north-south through the middle of the Atlantic Ocean. As tectonic plates pull away from each other at a divergent plate boundary, they create deep fissures, or cracks, in the crust. Molten rock, called magma, erupts through these cracks onto Earth’s surface. At the surface, the molten rock is called lava. It cools and hardens, forming rock. Divergent plate boundaries also occur in the continental crust. Volcanoes form at these boundaries, but less often than in ocean crust. That’s because continental crust is thicker than oceanic crust. This makes it more difficult for molten rock to push up through the crust. Many volcanoes form along convergent plate boundaries where one tectonic plate is pulled down beneath another at a subduction zone. The leading edge of the plate melts as it is pulled into the mantle, forming magma that erupts as volcanoes. When a line of volcanoes forms along a subduction zone, they make up a volcanic arc. The edges of the Pacific plate are long subduction zones lined with volcanoes. This is why the Pacific rim is called the “Pacific Ring of Fire.”"""

wrapper = textwrap.TextWrapper(width=150)
word_list = wrapper.wrap(text=text)
for element in word_list: 
  print(element) 

There is a lot of volcanic activity at divergent plate boundaries in the oceans. For example, many undersea volcanoes are found along the Mid-Atlantic
Ridge. This is a divergent plate boundary that runs north-south through the middle of the Atlantic Ocean. As tectonic plates pull away from each other
at a divergent plate boundary, they create deep fissures, or cracks, in the crust. Molten rock, called magma, erupts through these cracks onto Earth’s
surface. At the surface, the molten rock is called lava. It cools and hardens, forming rock. Divergent plate boundaries also occur in the continental
crust. Volcanoes form at these boundaries, but less often than in ocean crust. That’s because continental crust is thicker than oceanic crust. This
makes it more difficult for molten rock to push up through the crust. Many volcanoes form along convergent plate boundaries where one tectonic plate
is pulled down beneath another at a subduction zone. The leading edge of the plate melts as it is pu

In [3]:
import json
import requests
import string
import re
import nltk
import string
import itertools
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
import pke
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import traceback
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
def tokenize_sentences(text):
    sentences = sent_tokenize(text)
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

sentences = tokenize_sentences(text)
print (sentences)

['There is a lot of volcanic activity at divergent plate boundaries in the oceans.', 'For example, many undersea volcanoes are found along the Mid-Atlantic Ridge.', 'This is a divergent plate boundary that runs north-south through the middle of the Atlantic Ocean.', 'As tectonic plates pull away from each other at a divergent plate boundary, they create deep fissures, or cracks, in the crust.', 'Molten rock, called magma, erupts through these cracks onto Earth’s surface.', 'At the surface, the molten rock is called lava.', 'It cools and hardens, forming rock.', 'Divergent plate boundaries also occur in the continental crust.', 'Volcanoes form at these boundaries, but less often than in ocean crust.', 'That’s because continental crust is thicker than oceanic crust.', 'This makes it more difficult for molten rock to push up through the crust.', 'Many volcanoes form along convergent plate boundaries where one tectonic plate is pulled down beneath another at a subduction zone.', 'The leadi

In [5]:

def get_noun_adj_verb(text):
    out=[]
    try:
        extractor = pke.unsupervised.MultipartiteRank()
        extractor.load_document(input=text)
        pos = {'VERB', 'ADJ', 'NOUN'}
        stoplist = list(string.punctuation)
        stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
        stoplist += stopwords.words('english')
        extractor.candidate_selection(pos=pos, stoplist=stoplist)
       
        extractor.candidate_weighting(alpha=1.1,
                                      threshold=0.75,
                                      method='average')
        keyphrases = extractor.get_n_best(n=30)
        

        for val in keyphrases:
            out.append(val[0])
    except:
        out = []
        traceback.print_exc()

    return out

noun_verbs_adj = get_noun_adj_verb(text)
print ("keywords: ",noun_verbs_adj)

keywords:  ['divergent plate boundaries', 'many undersea volcanoes', 'crust', 'oceans', 'molten rock', 'tectonic plates pull', 'called', 'example', 'volcanic activity', 'cracks', 'found', 'forming rock', 'surface', 'subduction zone', 'erupts', 'continental crust', 'volcanoes form', 'ocean crust', 'runs', 'makes', 'pulled', 'lot', 'south', 'leading edge', 'hardens', 'middle', 'earth', 'create deep fissures', 'cools', 'volcanoes']


In [6]:
from pprint import pprint
def get_sentences_from_keywords(keywords,sentences):
  keyword_processor = KeywordProcessor()
  keyword_sentences = {}
  for word in keywords:
    keyword_sentences[word] = []
    keyword_processor.add_keyword(word)
  for sentence in sentences:
    keywords_found = keyword_processor.extract_keywords(sentence)
    for key in keywords_found:
      keyword_sentences[key].append(sentence)
  for key in keyword_sentences.keys():
    values = keyword_sentences[key]
    values = sorted(values,key=len,reverse=True)
    keyword_sentences[key] =values
  return keyword_sentences

keyword_sentence_mapping_noun_verb_adj = get_sentences_from_keywords(noun_verbs_adj,sentences)
pprint(keyword_sentence_mapping_noun_verb_adj)

{'called': ['Molten rock, called magma, erupts through these cracks onto '
            'Earth’s surface.',
            'This is why the Pacific rim is called the “Pacific Ring of Fire.”',
            'At the surface, the molten rock is called lava.'],
 'continental crust': ['Divergent plate boundaries also occur in the '
                       'continental crust.',
                       'That’s because continental crust is thicker than '
                       'oceanic crust.'],
 'cools': ['It cools and hardens, forming rock.'],
 'cracks': ['As tectonic plates pull away from each other at a divergent plate '
            'boundary, they create deep fissures, or cracks, in the crust.',
            'Molten rock, called magma, erupts through these cracks onto '
            'Earth’s surface.'],
 'create deep fissures': ['As tectonic plates pull away from each other at a '
                          'divergent plate boundary, they create deep '
                          'fissures, or cracks,

In [7]:
def get_fill_in_the_blanks(sentence_mapping):
  out = {'title':'Fill in the blanks for these sentences with matching words at the top'}
  blank_sentences = []
  processed = []
  keys = []
  for key in sentence_mapping:
    if len(sentence_mapping)>0:
      sent = sentence_mapping[key][0]
      insensitive_sent = re.compile(re.escape(key),re.IGNORECASE)
      no_of_replacements = len(re.findall(re.escape(key),sent,re.IGNORECASE))
      line = insensitive_sent.sub('_______',sent)
      if (sentence_mapping[key][0] not in processed) and no_of_replacements<2:
        blank_sentences.append(line)
        processed.append(sentence_mapping[key][0])
        keys.append(key)
        out['sentences'] = blank_sentences[:10]
        out['keys'] = keys[:10]
  
  return out

fill_in_the_blanks = get_fill_in_the_blanks(keyword_sentence_mapping_noun_verb_adj)

In [8]:
fill_in_the_blanks

{'keys': ['divergent plate boundaries',
  'many undersea volcanoes',
  'crust',
  'molten rock',
  'forming rock',
  'subduction zone',
  'erupts',
  'continental crust',
  'ocean crust',
  'runs'],
 'sentences': ['There is a lot of volcanic activity at _______ in the oceans.',
  'For example, _______ are found along the Mid-Atlantic Ridge.',
  'As tectonic plates pull away from each other at a divergent plate boundary, they create deep fissures, or cracks, in the _______.',
  '_______, called magma, erupts through these cracks onto Earth’s surface.',
  'It cools and hardens, _______.',
  'Many volcanoes form along convergent plate boundaries where one tectonic plate is pulled down beneath another at a _______.',
  'The leading edge of the plate melts as it is pulled into the mantle, forming magma that _______ as volcanoes.',
  'Divergent plate boundaries also occur in the _______.',
  'Volcanoes form at these boundaries, but less often than in _______.',
  'This is a divergent plate b

In [9]:
from IPython.core.display import display, HTML
import xml.etree.ElementTree as et
import random

root = et.Element("div")

heading = et.Element("h2")
heading.text = fill_in_the_blanks['title']

keywords = et.Element("ul")
keywords.set('style', 'color:blue;')

all_keys = fill_in_the_blanks['keys']
random.shuffle(all_keys)
for blank in all_keys:
  child=et.Element("li")
  child.text = blank
  keywords.append(child)

sentences = et.Element("ol")
sentences.set('style', 'color:brown;')
for sentence in fill_in_the_blanks['sentences']:
  child=et.Element("li")
  child.text = sentence
  sentences.append(child)
  sentences.append(et.Element("br"))

heading_content = et.Element("h4")

root.append(heading)
heading_content.append(keywords)
heading_content.append(sentences)
root.append(heading_content)

xmlstr = et.tostring(root)
xmlstr = xmlstr.decode("utf-8") 
display(HTML(xmlstr))